## 0. Imports and Setup

In [ ]:
import os
import sys
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt

# Install required packages if missing
try:
    import chainconsumer, seaborn, emcee, arviz
except ImportError:
    print('Installing required packages...')
    os.system('pip install seaborn emcee chainconsumer arviz')

# Auto-reload for development
%load_ext autoreload
%autoreload 2

# Import pipeline components
from scat_analysis.burstfit_pipeline import BurstPipeline
from scat_analysis.burstfit_corner import (
    quick_chain_check,
    get_clean_samples,
    make_beautiful_corner,
    make_beautiful_corner_wide
)
from scat_analysis.burstfit import FRBParams

## 1. Configuration

**Select your burst configuration file below**. All burst parameters (data path, telescope settings, MCMC parameters) are loaded from the YAML config.

In [ ]:
# ============================================================
# SELECT YOUR BURST CONFIGURATION
# ============================================================
config_path = Path("../configs/bursts/dsa/casey_dsa.yaml")

# Load configuration
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract key parameters
data_path = Path(config['path'])
burst_name = data_path.stem.split('_')[0]  # Extract burst name from filename
telescope = config['telescope']
dm_initial = config.get('dm_init', 0.0)

# Set output directory
plot_dir = Path(config.get('outpath', f'../plots/{telescope}'))
plot_dir.mkdir(parents=True, exist_ok=True)

print(f"Burst: {burst_name}")
print(f"Telescope: {telescope}")
print(f"Data: {data_path}")
print(f"Output: {plot_dir}")

## 2. Build and Run Pipeline

The pipeline performs:
- Data loading and preprocessing
- Initial guess optimization
- Model selection scan (BIC comparison of M0-M3 models)
- MCMC sampling with adaptive extension
- Comprehensive diagnostics (sub-band, influence analysis)
- Automated plot generation

In [ ]:
# Create pipeline instance
pipeline_params = {
    'telescope': config['telescope'],
    'telcfg_path': '../configs/telescopes.yaml',
    'sampcfg_path': '../configs/sampler.yaml',
    'steps': config.get('steps', 1000),
    'f_factor': config.get('f_factor', 384),
    't_factor': config.get('t_factor', 2),
    'center_burst': config.get('center_burst', True),
    'outer_trim': config.get('outer_trim', 0.49),
    'smooth_ms': config.get('smooth_ms', 0.1),
    'nproc': config.get('nproc', 16),
    'yes': True  # Auto-confirm pool creation
}

pipe = BurstPipeline(
    name=burst_name,
    inpath=data_path,
    outpath=plot_dir,
    dm_init=dm_initial,
    **pipeline_params
)

print(f"Pipeline created for {burst_name} ({telescope.upper()})")

In [ ]:
# Run full pipeline
results = pipe.run_full(
    model_scan=config.get('model_scan', True),
    model_keys=["M3"],
    diagnostics=config.get('diagnostics', True),
    plot=config.get('plot', True),
    save=True,
    show=False
)

print("\n" + "="*60)
print("PIPELINE RUN SUMMARY")
print("="*60)
print(f"Best model: {results['best_key']}")
print(f"Reduced χ²: {results['goodness_of_fit']['chi2_reduced']:.2f}")
print("\nBest-fit parameters:")
for param, value in results['best_params'].items():
    print(f"  {param}: {value:.4g}")

## 3. Interactive Post-Fit Analysis (Optional)

The pipeline automatically generates comprehensive plots. Use the cells below for additional interactive analysis:
- Extended MCMC chain convergence checking
- Custom corner plots
- Detailed parameter exploration

In [ ]:
# Extract results for interactive analysis
sampler = results["sampler"]
best_p = results["best_params"]
param_names = results["param_names"]

# Detach sampler from pool for serial operation
sampler.pool = None

# Check chain convergence
print("Checking MCMC chain convergence...")
max_extra_chunks = config.get('max_chunks', 2)
chunk_size = config.get('chunk_size', 100)
chunks_added = 0

if config.get('extend_chain', False):
    while not quick_chain_check(sampler):
        if chunks_added >= max_extra_chunks:
            print(f"Reached max extra steps ({max_extra_chunks * chunk_size}); proceeding.")
            break
        print(f"Running {chunk_size} more steps for convergence...")
        sampler.run_mcmc(None, chunk_size, progress=True)
        chunks_added += 1
    print(f"Chain converged after {chunks_added} additional chunks.")
else:
    converged = quick_chain_check(sampler)
    print(f"Chain convergence: {'PASSED' if converged else 'NOT CONVERGED'}")

In [ ]:
# Generate corner plot
print("Generating corner plot...")
final_clean_samples = get_clean_samples(sampler, param_names, verbose=True)

fig_corner = make_beautiful_corner_wide(
    final_clean_samples,
    param_names,
    best_params=best_p,
    title=f"Posterior for {results['best_key']} ({final_clean_samples.shape[0]} samples)"
)

corner_path = plot_dir / f"{burst_name}_scat_corner.pdf"
fig_corner.savefig(corner_path, dpi=300, bbox_inches='tight')
print(f"Saved corner plot to: {corner_path}")
plt.show()

## Analysis Complete

Results saved to: `{plot_dir}`

### Next Steps
- Review the 16-panel diagnostic plot: `{burst_name}_scat_fit.pdf`
- Check corner plot: `{burst_name}_scat_corner.pdf`
- Examine fit parameters in the summary above
- Adjust config file and re-run if needed